In [11]:
#버전확인
!nvcc --version 쿠다 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Fri_Dec_17_18:28:54_Pacific_Standard_Time_2021
Cuda compilation tools, release 11.6, V11.6.55
Build cuda_11.6.r11.6/compiler.30794723_0


In [9]:
#쿠다버전맞게설치
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://download.pytorch.org/whl/cu116


In [10]:
#파이토치 쿠다버전확인
import torch
print("CUDA 사용 가능:", torch.cuda.is_available())
print("CUDA 버전:", torch.version.cuda)
print("GPU 이름:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "GPU 없음")

CUDA 사용 가능: False
CUDA 버전: None
GPU 이름: GPU 없음


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# 하이퍼파라미터
batch_size = 32
epochs = 30
lr = 0.001
num_classes = 5
image_size = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 경로 설정
train_dir = "hazelnut/train"

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

# 데이터셋 로드
train_dataset = ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 간단한 CNN 모델 정의
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(32 * 56 * 56, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# 학습 루프
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch+1}/{epochs}] Loss: {running_loss / len(train_loader):.4f}")

# ONNX로 모델 저장
dummy_input = torch.randn(1, 3, image_size, image_size).to(device)
torch.onnx.export(model, dummy_input, "hazel_model.onnx",
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

print("✅ ONNX 모델 저장 완료: hazel_model.onnx")


[Epoch 1/30] Loss: 2.1626
[Epoch 2/30] Loss: 1.7312
[Epoch 3/30] Loss: 1.5541
[Epoch 4/30] Loss: 1.5329
[Epoch 5/30] Loss: 1.4922
[Epoch 6/30] Loss: 1.5015
[Epoch 7/30] Loss: 1.4190
[Epoch 8/30] Loss: 1.3393
[Epoch 9/30] Loss: 1.2411
[Epoch 10/30] Loss: 1.0893
[Epoch 11/30] Loss: 1.1639
[Epoch 12/30] Loss: 0.9954
[Epoch 13/30] Loss: 0.9132
[Epoch 14/30] Loss: 0.8854
[Epoch 15/30] Loss: 0.7562
[Epoch 16/30] Loss: 0.6999
[Epoch 17/30] Loss: 0.6221
[Epoch 18/30] Loss: 0.5377
[Epoch 19/30] Loss: 0.5350
[Epoch 20/30] Loss: 0.5271
[Epoch 21/30] Loss: 0.4357
[Epoch 22/30] Loss: 0.3977
[Epoch 23/30] Loss: 0.3086
[Epoch 24/30] Loss: 0.3469
[Epoch 25/30] Loss: 0.3349
[Epoch 26/30] Loss: 0.3268
[Epoch 27/30] Loss: 0.2329
[Epoch 28/30] Loss: 0.3436
[Epoch 29/30] Loss: 0.2250
[Epoch 30/30] Loss: 0.2042
✅ ONNX 모델 저장 완료: hazel_model.onnx


In [23]:
print(train_dataset.class_to_idx)

{'crack': 0, 'cut': 1, 'good': 2, 'hole': 3, 'print': 4}


## Epoch 10으로하니까 25퍼 나옴
## 30으로 변경해서 사용 5퍼 나옴

In [22]:
import cv2
import numpy as np
from glob import glob
import os

# 클래스 인덱스 기준 (PyTorch에서 학습한 기준)
class_names = ['crack', 'cut', 'good', 'hole', 'print']
GOOD_INDEX = 2  # good 클래스 인덱스

# 모델 불러오기
net = cv2.dnn.readNetFromONNX("hazel_model.onnx")

# 테스트 이미지 경로 (good 클래스만)
image_paths = glob("hazelnut/test/good/*.png")  # .jpg면 바꿔줘

correct = 0
total = 0
pred_counts = [0] * 5  # 클래스별 예측 개수 기록

for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        print(f"이미지 로드 실패: {path}")
        continue

    blob = cv2.dnn.blobFromImage(img, 1.0/255.0, (224, 224), swapRB=True)
    net.setInput(blob)
    output = net.forward()

    pred = np.argmax(output)
    pred_counts[pred] += 1
    total += 1

    print(f"[{total}] {os.path.basename(path)} → 예측: {class_names[pred]} ({pred})")

    if pred == GOOD_INDEX:
        correct += 1

# 정확도 출력
accuracy = correct / total * 100 if total > 0 else 0
print(f"\n✅ good 테스트 정확도: {accuracy:.2f}% ({correct}/{total})")

# 예측 분포 출력
print("\n📊 클래스별 예측 개수:")
for i, count in enumerate(pred_counts):
    print(f"{class_names[i]} ({i}): {count}")

[1] 000.png → 예측: print (4)
[2] 001.png → 예측: print (4)
[3] 002.png → 예측: print (4)
[4] 003.png → 예측: print (4)
[5] 004.png → 예측: print (4)
[6] 005.png → 예측: hole (3)
[7] 006.png → 예측: print (4)
[8] 007.png → 예측: print (4)
[9] 008.png → 예측: hole (3)
[10] 009.png → 예측: print (4)
[11] 010.png → 예측: hole (3)
[12] 011.png → 예측: hole (3)
[13] 012.png → 예측: print (4)
[14] 013.png → 예측: hole (3)
[15] 014.png → 예측: hole (3)
[16] 015.png → 예측: hole (3)
[17] 016.png → 예측: hole (3)
[18] 017.png → 예측: print (4)
[19] 018.png → 예측: print (4)
[20] 019.png → 예측: print (4)
[21] 020.png → 예측: print (4)
[22] 021.png → 예측: hole (3)
[23] 022.png → 예측: hole (3)
[24] 023.png → 예측: print (4)
[25] 024.png → 예측: print (4)
[26] 025.png → 예측: print (4)
[27] 026.png → 예측: hole (3)
[28] 027.png → 예측: hole (3)
[29] 028.png → 예측: print (4)
[30] 029.png → 예측: print (4)
[31] 030.png → 예측: hole (3)
[32] 031.png → 예측: hole (3)
[33] 032.png → 예측: hole (3)
[34] 033.png → 예측: hole (3)
[35] 034.png → 예측: print (4)
[36] 035.p

[285] 284.png → 예측: print (4)
[286] 285.png → 예측: print (4)
[287] 286.png → 예측: print (4)
[288] 287.png → 예측: print (4)
[289] 288.png → 예측: print (4)
[290] 289.png → 예측: print (4)
[291] 290.png → 예측: print (4)
[292] 291.png → 예측: print (4)
[293] 292.png → 예측: print (4)
[294] 293.png → 예측: print (4)
[295] 294.png → 예측: print (4)
[296] 295.png → 예측: print (4)
[297] 296.png → 예측: print (4)
[298] 297.png → 예측: print (4)
[299] 298.png → 예측: print (4)
[300] 299.png → 예측: print (4)
[301] 300.png → 예측: print (4)
[302] 301.png → 예측: print (4)
[303] 302.png → 예측: print (4)
[304] 303.png → 예측: print (4)
[305] 304.png → 예측: print (4)
[306] 305.png → 예측: print (4)
[307] 306.png → 예측: print (4)
[308] 307.png → 예측: print (4)
[309] 308.png → 예측: print (4)
[310] 309.png → 예측: print (4)
[311] 310.png → 예측: print (4)
[312] 311.png → 예측: print (4)
[313] 312.png → 예측: print (4)
[314] 313.png → 예측: print (4)
[315] 314.png → 예측: print (4)
[316] 315.png → 예측: print (4)
[317] 316.png → 예측: print (4)
[318] 317.

# ResNet18 기반 분류 모델 전체 코드 (PyTorch)

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torch.utils.data import DataLoader
import os

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 하이퍼파라미터
num_classes = 5
batch_size = 32
epochs = 10
lr = 0.001

# 경로
train_dir = "hazelnut/train"

# 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 데이터셋 및 데이터로더
train_dataset = ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 모델 정의 (ResNet18 사전학습 모델 사용)
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# 학습 루프
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = correct / total * 100
    print(f"[Epoch {epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f} | Accuracy: {acc:.2f}%")

# ONNX로 저장
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(model, dummy_input, "hazel_model.onnx",
                  input_names=["input"], output_names=["output"],
                  dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print("✅ ONNX 모델 저장 완료: hazel_model.onnx")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\AN/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

[Epoch 1/10] Loss: 1.1120 | Accuracy: 48.18%
[Epoch 2/10] Loss: 0.3878 | Accuracy: 87.27%
[Epoch 3/10] Loss: 0.3741 | Accuracy: 90.91%
[Epoch 4/10] Loss: 0.2438 | Accuracy: 91.82%
[Epoch 5/10] Loss: 0.0596 | Accuracy: 97.27%
[Epoch 6/10] Loss: 0.0427 | Accuracy: 99.09%
[Epoch 7/10] Loss: 0.0187 | Accuracy: 100.00%
[Epoch 8/10] Loss: 0.0076 | Accuracy: 100.00%
[Epoch 9/10] Loss: 0.0080 | Accuracy: 100.00%
[Epoch 10/10] Loss: 0.1897 | Accuracy: 98.18%
✅ ONNX 모델 저장 완료: hazel_model.onnx


# ResNet18으로 바꾸고 정확도 100퍼